In [2]:
import csv
import requests
from datetime import datetime
from time import mktime
import json

def collection(time, step, source, filename):
    """
    time: time in hours
    step: query interval within the timeframe
    source: json file with the metrics to collect
    filename: created csv file will have this name
    """
    now = datetime.now()
    now_unix = mktime(now.timetuple())
    file = open(source)
    metrics = json.load(file)['data']

    #quick'n dirty bool to add timestamps to only first iteration
    first = True
    for metric in metrics:
        response = requests.get(
            'http://localhost:9090/api/v1/query_range',
            params={
                'query': metric,
                'start': now_unix - 3600 * time,
                "end": now_unix - 3600* (time - 3),
                'step': step
            }
        )


        results = response.json()['data']['result']
        print(results)

        

        labelnames = set()
        for result in results:
            labelnames.update(result['metric'].keys())
        labelnames = sorted(labelnames)
        
        with open(filename + ".csv", mode = "a") as f:
            writer = csv.writer(f, lineterminator='\n')
            if(first):
                first = False
                timestamps = []
                for x in results[0]['values']:
                    timestamps.append(x[0])
                writer.writerow(["identifier"] + timestamps)
            
            for result in results:
                vals = [x[1] for x in result['values']]
                identifier_string = ""
                for label in labelnames: 
                    identifier_string += result['metric'].get(label, '') + '&'
                    
                writer.writerow([identifier_string[:-1]] + vals)



collection(6, "10m", "metrics.json", "carts-1_cpu=1_time=2h")


[]


IndexError: list index out of range